In [ ]:
with open('inputs/day07.txt') as f:
    lines = [list(0 if c == '.' else c for c in l.strip()) for l in f.readlines()]

part1_ans = 0
part2_ans = 0
for i, line in enumerate(lines):
    if i == 0:
        continue

    for j, c in enumerate(line):
        if lines[i-1][j] == 'S':
            line[j] += 1
        elif lines[i-1][j] != '^' and lines[i-1][j] > 0:
            if c == '^':
                part1_ans += 1
                line[j-1] += lines[i-1][j]
                line[j+1] += lines[i-1][j]
            else:
                line[j] += lines[i-1][j]

print('Answer to Day 7, Part 1:', part1_ans)
print('Answer to Day 7, Part 2:', sum(lines[-1]))

In [ ]:
import itertools
import bisect
import math
import networkx as nx

with open('inputs/day08.txt') as f:
    positions = [tuple(int(x) for x in l.split(',')) for l in f.readlines()]

distances = []
for p1, p2 in itertools.product(positions, repeat=2):
    if p1 < p2:
        bisect.insort(distances, (math.dist(p1, p2), p1, p2))

g = nx.Graph()
g.add_nodes_from(range(len(positions)))
for edge in distances[:1000]:
    g.add_edge(positions.index(edge[1]), positions.index(edge[2]))

part1_ans = 1
for c in sorted(nx.connected_components(g), key=len, reverse=True)[:3]:
    part1_ans *= len(c)

print('Answer to Day 8, Part 1:', part1_ans)

g = nx.Graph()
g.add_nodes_from(range(len(positions)))
for edge in distances:
    g.add_edge(positions.index(edge[1]), positions.index(edge[2]))
    if nx.is_connected(g):
        print('Answer to Day 8, Part 2:', edge[1][0] * edge[2][0])
        break

In [ ]:
import itertools

with open('inputs/day09.txt') as f:
    positions = [tuple(int(x) for x in l.split(',')) for l in f.readlines()]

part1_ans = 0
for p1, p2 in itertools.product(positions, repeat=2):
    if p1 < p2:
        area = (abs(p1[0] - p2[0]) + 1) * (abs(p1[1] - p2[1]) + 1)
        part1_ans = max(part1_ans, area)

print('Answer to Day 9, Part 1:', part1_ans)

# I manually determined that the out-of-bounds sections are to the right of the given route
out_of_bounds_lines = []
for i in range(len(positions)):
    if positions[i][0] == positions[i-1][0]:
        if positions[i][1] > positions[i-1][1]:
            p1 = (positions[i-1][0] + 1, positions[i-1][1] + 1)
            p2 = (positions[i][0] + 1, positions[i][1] - 1)
            if i > 0 and p1 != out_of_bounds_lines[-1][1]:
                p1 = (p1[0], p1[1] - 2)

            out_of_bounds_lines.append((p1, p2))

        else:
            p1 = (positions[i-1][0] - 1, positions[i-1][1] - 1)
            p2 = (positions[i][0] - 1, positions[i][1] + 1)
            if i > 0 and p1 != out_of_bounds_lines[-1][1]:
                p1 = (p1[0], p1[1] + 2)
            
            out_of_bounds_lines.append((p1, p2))
    else:
        if positions[i][0] > positions[i-1][0]:
            p1 = (positions[i-1][0] + 1, positions[i-1][1] - 1)
            p2 = (positions[i][0] - 1, positions[i][1] - 1)
            if i > 0 and p1 != out_of_bounds_lines[-1][1]:
                p1 = (p1[0] - 2, p1[1])

            out_of_bounds_lines.append((p1, p2))
        else:
            p1 = (positions[i-1][0] - 1, positions[i-1][1] + 1)
            p2 = (positions[i][0] + 1, positions[i][1] + 1)
            if i > 0 and p1 != out_of_bounds_lines[-1][1]:
                p1 = (p1[0] + 2, p1[1])

            out_of_bounds_lines.append((p1, p2))

def lines_intersect(line1, line2):
    (x1, y1), (x2, y2) = line1
    (x3, y3), (x4, y4) = line2
    denom = (y4 - y3) * (x2 - x1) - (x4 - x3) * (y2 - y1)
    if denom == 0: # parallel
        return False

    ua = ((x4-x3)*(y1-y3) - (y4-y3)*(x1-x3)) / denom
    if ua < 0 or ua > 1: # out of range
        return False

    ub = ((x2-x1)*(y1-y3) - (y2-y1)*(x1-x3)) / denom
    if ub < 0 or ub > 1: # out of range
        return False

    return True

part2_ans = 0
for p1, p2 in itertools.product(positions, repeat=2):
    if p1 < p2:
        area = (abs(p1[0] - p2[0]) + 1) * (abs(p1[1] - p2[1]) + 1)
        if area > part2_ans and not any(
            lines_intersect(p, (p1, (p1[0], p2[1]))) or
            lines_intersect(p, (p1, (p2[0], p1[1]))) or
            lines_intersect(p, (p2, (p1[0], p2[1]))) or
            lines_intersect(p, (p2, (p2[0], p1[1])))
            for p in out_of_bounds_lines
        ):
            part2_ans = area

print('Answer to Day 9, Part 2:', part2_ans)

In [ ]:
import numpy as np
import itertools
from ortools.linear_solver import pywraplp

machines = []
with open('inputs/day10.txt') as f:
    for line in f.readlines():
        stuff = line.split(' ')
        lights = np.array([int(c=='#') for c in stuff[0][1:-1]])
        machines.append({
            'lights': lights,
            'buttons': [np.array([int(str(i) in b[1:-1].split(',')) for i in range(len(lights))]) for b in stuff[1:-1]],
            'joltages': list(map(int, stuff[-1].strip()[1:-1].split(',')))
        })

def get_button_count(machine):
    for i in range(len(machine['buttons'])):
        for things in itertools.combinations(machine['buttons'], r=i):
            if np.all(sum(things) % 2 == machine['lights']):
                return i

print('Answer to Day 10, Part 1:', sum(get_button_count(m) for m in machines))

def get_button_count_pt2(machine):
    # Create the mip solver with the CP-SAT backend.
    solver = pywraplp.Solver.CreateSolver("SAT")

    vars = [
        solver.IntVar(0, max(machine['joltages']), f'x{i}')
        for i in range(len(machine['buttons']))
    ]

    for i, joltage in enumerate(machine['joltages']):
        solver.Add(
            sum(vars[j] for j, button in enumerate(machine['buttons']) if button[i]) == joltage
        )

    solver.Minimize(sum(vars))
    solver.Solve()
    return int(solver.Objective().Value())

print('Answer to Day 10, Part 2:', sum(get_button_count_pt2(m) for m in machines))